## Segmenting and Clustering Neighborhoods in Toronto

# PART 1

##### Install libraries

In [1]:
# install beautifulsoup4 library using to parse HTML page

!pip install beautifulsoup4
!pip install html5lib
!pip install geopy
!pip install folium==0.5.0

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
# map rendering library
import folium 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Complete imported libraries')

Complete imported libraries


##### Using BeautifulSoup library to scrape the Wikipedia page to get Postal Code

In [2]:
# parsing the html to get Postal_codes_in_Canada
def getPostalCode_DataFrame(page_content):

    soup = BeautifulSoup(page_content, 'html.parser')

    #print(soup.prettify())
    # find the table that contains Postal_codes_in_Canada
    tbl = (soup.find_all(class_='wikitable sortable'))


    # if the table exists then parsing the rows.
    if (len(tbl) >0):
        rows = tbl[0].find_all('tr')

        col1 = pd.DataFrame(columns = ["PostalCode"])
        col2 = pd.DataFrame(columns = ["Borough"])
        col3 = pd.DataFrame(columns = ["Neighborhood"])

        # loop through rows
        for  i, row in enumerate(rows): 

            # skip the header, get data rows
            if (i>0):
                columns = row.find_all('td') 

                # there are 3 columns
                postalcode = str(columns[0]).replace('<td>','').replace('</td>','').replace('\n','').strip()
                borough = str(columns[1]).replace('<td>','').replace('</td>','').replace('\n',''). strip()
                neighborhoods =  str(columns[2]).replace('<td>','').replace('</td>','').replace('\n','').strip()
              
                col1.loc[i-1] = postalcode 
                col2.loc[i-1] = borough
                col3.loc[i-1] = neighborhoods
                    
                df_raw = pd.concat([col1, col2, col3],  axis=1)
    
        return df_raw
    else:
        return None

In [27]:
# request html page
html = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(html)
page_content = page.content


##### Cleansing data and combine postal code and geo code data frames to create a final data frame

In [30]:
# cleansing postal code dataframe

# get postal code dataframe from parsing function
df_raw = getPostalCode_DataFrame(page_content)

#print(df_raw.dtypes)
#print(df_raw.shape)
#print(df_raw.Borough.unique())

# removed Borough = 'Not assigned'
df_postal = df_raw[df_raw.Borough != 'Not assigned'].copy()

#print(df_postal.shape)  #removed 77 Borough = 'Not assigned'

# borough has value but neighborhood has Not assigned. Set neighborhood to be the same as the borough.
# check Neighborhood = 'Not assigned' in the df_postal. Since Borough = 'Not assigned' was removed

# it is safe to assume Borough is valid and should assign to 'Not assigned' Neighbourhood
# replace Neighborhood = 'Not assigned' to NaN
df_postal.Neighborhood = df_postal.Neighborhood.replace('Not assigned',np.NaN)

# then replace NaN with Borough's value
df_postal['Neighborhood'].fillna(df_postal['Borough'], inplace=True)

df_postal.reset_index(drop=True,inplace = True)

df_postal.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# PART 2

#### Get Latitude and Longitude from Geospatial_data and combine with Postal code data frame

In [33]:
# load geographical coordinates of each postal code
url = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url, index_col=None, header='infer')

df_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_geo.set_index('PostalCode',inplace=True)

#df_geo.head(15)

In [34]:
# combine dataframe

df_postal.set_index('PostalCode',inplace=True)

neighborhoods = df_postal.merge(df_geo, left_index=True, right_index=True)

# reset index
neighborhoods.reset_index(inplace=True)

neighborhoods.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [35]:
neighborhoods.shape

(103, 5)

# PART 3

## Explore and cluster the neighborhoods in Toronto.
##### To minimize Foursquare call. Get venues for neighborhoods around Toronto Downtown, East Toronto, West Toronto, and Central Toronto

In [36]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.shape

(39, 5)

In [37]:
# The code was removed by Watson Studio for sharing.

In [38]:
# using function from the lecture
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

###### Run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [39]:
toronto_venues = getNearbyVenues(toronto_data["Neighborhood"], toronto_data["Latitude"], toronto_data["Longitude"])

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped.shape

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.head() # check the last columns!

##### Create map to show Toronto neighborhood clusters

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters